In [1]:
import cv2
import mediapipe as mp
import pyautogui
import time
import datetime

2024-03-02 17:56:25.979014: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def count_fingers(lst):
    cnt=0
    
    thresh = (lst.landmark[0].y*100-lst.landmark[9].y*100)/2
    
    if(lst.landmark[5].y*100-lst.landmark[8].y*100)>thresh:
        cnt+=1
    if(lst.landmark[9].y*100-lst.landmark[12].y*100)>thresh:
        cnt+=1
    if(lst.landmark[13].y*100-lst.landmark[16].y*100)>thresh:
        cnt+=1
    if(lst.landmark[17].y*100-lst.landmark[20].y*100)>thresh:
        cnt+=1
    if(lst.landmark[5].y*100-lst.landmark[4].y*100)>5:
        cnt+=1
    return cnt

def rescale(frame,scale=0.2):
    width = int(2*frame.shape[1]*scale)
    height = int(2*frame.shape[0]*scale)
    dimension = (width,height)
    return cv2.resize(frame,dimension,interpolation=cv2.INTER_AREA)

cap = cv2.VideoCapture(0)
drawing = mp.solutions.drawing_utils
hands = mp.solutions.hands
hand_obj = hands.Hands(max_num_hands=2)

start_init = False
prev = -1

while True:
    end_time = time.time()
    ret,frm = cap.read()
    frm = cv2.flip(frm,1)
    frm = cv2.resize(frm,(1000,800))
    frm = cv2.copyMakeBorder(frm,15,15,15,15,cv2.BORDER_CONSTANT,value=0)
     
    res = hand_obj.process(cv2.cvtColor(frm,cv2.COLOR_BGR2RGB))
    
    if ret==True:
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        text = '   Height: ' + str(cap.get(4)) + '   Width: ' + str(cap.get(3))
        date_data = "Date: "+ str(datetime.datetime.now())
        frame = cv2.putText(frm,text,(20,50),font,2,(0,0,0),2)
        frame = cv2.putText(frm,date_data,(60,90),font,2,(0,0,0),2)
    
    if res.multi_hand_landmarks:
        
        hand_keyPoints = res.multi_hand_landmarks[0]
        cnt = count_fingers(hand_keyPoints)
        print("Identified Number: ",cnt)
        if(cnt==1):
            print("Fast Forward 5 seconds")
        elif(cnt==2):
            print("Rewind 5 seconds")
        elif(cnt==3):
            print("Increased Volume by 5%")
        elif(cnt==4):
            print("Decreased Volume by 5%")
        elif(cnt==5):
            print("Start/Pause Video")
        else:
            print("No operation")

        if not(prev==cnt):
            if not(start_init):
                start_time = time.time()
                start_init = True
            elif (end_time-start_time)>0.3:
                if(cnt==1):
                    pyautogui.press("right")
                elif (cnt==2):
                    pyautogui.press("left")
                elif (cnt==3):
                    pyautogui.press("up")
                elif (cnt==4):
                    pyautogui.press("down")
                elif (cnt==5):
                    pyautogui.press("space")
                prev = cnt
                start_init = False
        drawing.draw_landmarks(frm,hand_keyPoints,hands.HAND_CONNECTIONS)
        
    cv2.imshow('Hand Gesture Screen', rescale(frm))
    
    if cv2.waitKey(1) == 27:
        cv2.destroyAllWindows()
        cap.release()
        cv2.waitKey(0)
        break

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Identified Number:  3
Increased Volume by 5%
Identified Number:  2
Rewind 5 seconds
Identified Number:  3
Increased Volume by 5%
Identified Number:  3
Increased Volume by 5%
Identified Number:  3
Increased Volume by 5%
Identified Number:  3
Increased Volume by 5%
Identified Number:  3
Increased Volume by 5%
Identified Number:  3
Increased Volume by 5%
Identified Number:  0
No operation
Identified Number:  2
Rewind 5 seconds
Identified Number:  2
Rewind 5 seconds
Identified Number:  2
Rewind 5 seconds
Identified Number:  3
Increased Volume by 5%
Identified Number:  0
No operation
Identified Number:  1
Fast Forward 5 seconds
Identified Number:  0
No operation
Identified Number:  2
Rewind 5 seconds
Identified Number:  5
Start/Pause Video
Identified Number:  5
Start/Pause Video
Identified Number:  4
Decreased Volume by 5%
Identified Number:  4
Decreased Volume by 5%
Identified Number:  4
Decreased Volume by 5%
Identified Number:  5
Start/Pause Video
Identified Number:  5
Start/Pause Video
